# Prequesite

In [239]:
! python -m venv .venv
! source .venv/bin/activate

In [240]:
! pip install --upgrade pip
! pip install ipykernel
! python -m ipykernel install --name=.venv
! pip install numpy pandas
! pip freeze > requirements.txt

Installed kernelspec .venv in /usr/local/share/jupyter/kernels/.venv


In [241]:
import json
import numpy as np 
import pandas as pd
import scipy as sc
from domain import CSRInquiries, ShiftsDetail, Schedule
from functools import reduce


# Input

In [242]:
# Incoming customer inquiries
csr_inquiries = CSRInquiries.read_json('data/json/days.json')
csr_inquiries.to_dataframe()

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,6,6,7,6,6,6,5
1,9,10,9,9,7,9,7
2,9,7,9,8,8,9,6
3,8,7,6,6,7,4,5
4,3,3,3,4,3,3,4
5,3,4,4,4,5,3,3
6,7,7,6,5,6,4,4
7,8,5,8,8,7,5,5
8,8,9,7,7,6,5,6
9,5,5,4,5,5,5,5


In [243]:
# List of shifts
shifts_detail = ShiftsDetail.read_json('data/json/shifts.json')
shifts_detail.to_dataframe()

,C1,C2,C3,C4,C5,C6
0,1,0,0,0,0,1
1,1,1,0,0,0,1
2,1,1,1,0,0,1
3,1,1,1,1,0,1
4,0,0,1,1,1,0
5,1,1,0,1,1,0
6,1,1,1,1,1,0
7,1,1,1,1,1,0
8,1,1,1,0,1,0
9,0,1,1,1,0,1


# Determine number of CSRs in a day 

In [244]:
def constraint_min_CSR_of_a_period(csr_inquiries: CSRInquiries, shifts_detail: ShiftsDetail, at:int):

    constraint_matrix = -1 * shifts_detail._shifts_detail

    constraint_vector = -1 * csr_inquiries._csr_inquiries[at]

    return constraint_matrix.T, constraint_vector


In [245]:
coefficients_c = np.ones(shifts_detail.num_of_shifts)
coefficients_c

array([1., 1., 1., 1., 1., 1.])

In [246]:
def min_csr_of_a_day(csr_inquiries: CSRInquiries, shifts_detail: ShiftsDetail ,day_j:int):
    constraint_matrix, constraint_vector = constraint_min_CSR_of_a_period(csr_inquiries,shifts_detail, day_j)
    return sc.optimize.linprog(coefficients_c, constraint_matrix, constraint_vector, integrality=3)

In [247]:
def show_pandas(result, col, row):
    stat = pd.DataFrame(np.array(result), columns=col, index=row, dtype=int) 
    stat["Total"] = stat.sum(axis=1) 
    return stat

In [248]:
def label_shift_to_employee(row: pd.Series, min_csr: int):
    label = [[name] * x for x, name in zip(row, row.index[:-1])] + [(min_csr - row[-1]) * [None]]
    return pd.Series(reduce(lambda pre, cur: pre + cur, label, []))

In [249]:
q1_result = [min_csr_of_a_day(csr_inquiries,shifts_detail,j).x for j in range(csr_inquiries.num_of_days)]
q1_show = show_pandas(q1_result, shifts_detail.columns, csr_inquiries.columns)
q1_show

,C1,C2,C3,C4,C5,C6,Total
Monday,7,0,1,2,0,2,12
Tuesday,7,0,2,1,0,3,13
Wednesday,6,0,1,2,0,3,12
Thursday,6,0,0,3,1,3,13
Friday,4,0,2,1,0,3,10
Saturday,5,0,0,3,0,4,12
Sunday,5,0,0,3,1,2,11


In [250]:
q1_csr = q1_show["Total"].max()
q1_csr

13

In [251]:
q4_base = [q * 2 for q in q1_result[:5]] + q1_result[5:]
q4_show: pd.DataFrame = show_pandas(q4_base, shifts_detail.columns, csr_inquiries.columns)
q4_show

,C1,C2,C3,C4,C5,C6,Total
Monday,14,0,2,4,0,4,24
Tuesday,14,0,4,2,0,6,26
Wednesday,12,0,2,4,0,6,24
Thursday,12,0,0,6,2,6,26
Friday,8,0,4,2,0,6,20
Saturday,5,0,0,3,0,4,12
Sunday,5,0,0,3,1,2,11


In [252]:
q4_csr = q4_show["Total"].max()
q4_csr

26

In [253]:
def convert_num_to_schedule_matrix():
    q4_schedule = q4_show.apply(lambda row : label_shift_to_employee(row, q4_csr), axis=1).set_axis([f"NV{i}" for i in range(1, q4_csr + 1)], axis=1).T
    return q4_schedule

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
NV1,C1,C1,C1,C1,C1,C1,C1
NV2,C1,C1,C1,C1,C1,C1,C1
NV3,C1,C1,C1,C1,C1,C1,C1
NV4,C1,C1,C1,C1,C1,C1,C1
NV5,C1,C1,C1,C1,C1,C1,C1
NV6,C1,C1,C1,C1,C1,C4,C4
NV7,C1,C1,C1,C1,C1,C4,C4
NV8,C1,C1,C1,C1,C1,C4,C4
NV9,C1,C1,C1,C1,C3,C6,C5
NV10,C1,C1,C1,C1,C3,C6,C6


# Determine number of CSRs in a day

### Theory 1: Using shift and a following condition

First, create a column to retain the amount of empty periods 

In [254]:
q2_show = q1_show.copy()
q2_csr = q1_csr
q2_show["Empty_Slot"] = q2_csr - q2_show["Total"]
q2_show

,C1,C2,C3,C4,C5,C6,Total,Empty_Slot
Monday,7,0,1,2,0,2,12,1
Tuesday,7,0,2,1,0,3,13,0
Wednesday,6,0,1,2,0,3,12,1
Thursday,6,0,0,3,1,3,13,0
Friday,4,0,2,1,0,3,10,3
Saturday,5,0,0,3,0,4,12,1
Sunday,5,0,0,3,1,2,11,2


Rearrage by empty slots

In [ ]:
q4_show["Empty_Slot"] = q4_csr - q4_show["Total"]
q4_show

,C1,C2,C3,C4,C5,C6,Total,Empty_Slot
Monday,14,0,2,4,0,4,24,2
Tuesday,14,0,4,2,0,6,26,0
Wednesday,12,0,2,4,0,6,24,2
Thursday,12,0,0,6,2,6,26,0
Friday,8,0,4,2,0,6,20,6
Saturday,5,0,0,3,0,4,12,14
Sunday,5,0,0,3,1,2,11,15


In [ ]:
q4_schedule["Sunday"] = q4_schedule["Sunday"].shift(q4_show.loc["Saturday", "Total"])
q4_schedule

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
NV1,C1,C1,C1,C1,C1,C1,None
NV2,C1,C1,C1,C1,C1,C1,None
NV3,C1,C1,C1,C1,C1,C1,None
NV4,C1,C1,C1,C1,C1,C1,None
NV5,C1,C1,C1,C1,C1,C1,None
NV6,C1,C1,C1,C1,C1,C4,None
NV7,C1,C1,C1,C1,C1,C4,None
NV8,C1,C1,C1,C1,C1,C4,None
NV9,C1,C1,C1,C1,C3,C6,None
NV10,C1,C1,C1,C1,C3,C6,None
